When we discussed the k-means algorithm, we saw that we had to give the number of clusters
as one of the input parameters. In the real world, we wouldn't have this information available.
We can definitely sweep the parameter space to find out the optimal number of clustes using
the silhouette coefficient score, but tht would be an expensive process! Wouldn't it be nice
fi there were a method that can just tell us he number of clusters in our data? This is where
**Density-Based Spatial Clustering of Applications with Noise (DBSCAN)** comes into the picture.

This works by treating datapoints as groups of dense clusters. If a point belongs to a cluster,
then there should be a lot of other points that belong to the same cluster. One of the
parameters that we can control is the maximum distance of this point from other points. This
is called **epsilon**. No two points in a given cluster should be further away that epsilon. You
can learn more about it at https://en.wikipedia.org/wiki/DBSCAN. One of the main advantages of 
this method is that it can deal with outliers. If there are some points located alone in a 
low-density area, DBSCAN will detect these points as outliers as opposed to forcing them into
a cluster.

In [ ]:
# import necessary packages
from itertools import cycle

import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
import matplotlib.pyplot as plt

from utilities import load_data

In [ ]:
# load the input data
input_file = 'data_perf.txt'
X = load_data(input_file)

In [ ]:
# initialize variables to find the best epsilon
eps_grid = np.linspace(0.3, 1.2, num=10)
silhouette_scores = []
eps_best = eps_grid[0]
silhouette_score_max = -1
model_best = None
labels_best = None

In [ ]:
for eps in eps_grid:
    # Train DBSCAN clustering model
    model = DBSCAN(eps=eps, min_samples=5).fit(X)

    # Extract labels
    labels = model.labels_

    # Extract performance metric 
    silhouette_score = round(metrics.silhouette_score(X, labels), 4)
    silhouette_scores.append(silhouette_score)

    print ("Epsilon: %.1f  --> silhouette score: %.4f"%(eps, silhouette_score))

    # Store the best score and its associated epsilon value
    if silhouette_score > silhouette_score_max:
        silhouette_score_max = silhouette_score
        eps_best = eps
        model_best = model
        labels_best = labels
        
# Best params
print ("\nBest epsilon = %0.1f"%(eps_best))

# Associated model and labels for best epsilon
model = model_best 
labels = labels_best

# Check for unassigned datapoints in the labels
offset = 0
if -1 in labels:
    offset = 1

# Number of clusters in the data 
num_clusters = len(set(labels)) - offset 

print ("\nEstimated number of clusters =", num_clusters)

In [ ]:
# Plot silhouette scores vs epsilon
plt.figure()
plt.bar(eps_grid, silhouette_scores, width=0.05, color='k', align='center')
plt.title('Silhouette score vs epsilon')

In [ ]:
# Extracts the core samples from the trained model
mask_core = np.zeros(labels.shape, dtype=np.bool)
mask_core[model.core_sample_indices_] = True

In [ ]:
# Plot resultant clusters 
plt.figure()
labels_uniq = set(labels)
markers = cycle('vo^s<>')
for cur_label, marker in zip(labels_uniq, markers):
    # Use black dots for unassigned datapoints
    if cur_label == -1:
        marker = '.'

    # Create mask for the current label
    cur_mask = (labels == cur_label)

    cur_data = X[cur_mask & mask_core]
    plt.scatter(cur_data[:, 0], cur_data[:, 1], marker=marker,
             edgecolors='black', s=96, facecolors='none')

    cur_data = X[cur_mask & ~mask_core]
    plt.scatter(cur_data[:, 0], cur_data[:, 1], marker=marker,
             edgecolors='black', s=32)

plt.title('Data separated into clusters')
plt.show()